In [53]:
from openai import AzureOpenAI

import os
import sys
import json
import re
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [2]:
sys.path.append("../")

# Now you can import util or any of its functions directly
from util import mock_data_to_txt

In [3]:
v = mock_data_to_txt()

In [ ]:
v[5][0]

In [4]:
client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_ENDPOINT"),
    api_version=os.getenv("OPENAI_API_VERSION"),
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [5]:
instructions = """You are a helpful assistant, that helps to create synthetic data. 
                  You will be provided with a medical note of a patient and a summary.
                  The description and the summary are both in Swedish, and you should
                  answer in Swedish.
                  Please generate another note-summary pair for a new, realistic fake patient.
                  Remember to use the same template as the provided example for the summary.
                  """

In [ ]:
responses = []

In [14]:
for i,(x,y) in enumerate(v[5:]):
    
    now = datetime.now()
    one_hour_later = now + timedelta(hours=1)
    current_time = one_hour_later.strftime("%H:%M:%S")
    
    print(i,"-",current_time)
    
    question = f"""Please create a similar patient <Description>{x}</Description><Summary>{y}</Summary>""" + \
    """Respond with the same template, using tags."""
    
    messages = [{"role": "system", "content": instructions}, {"role": "user", "content": question}]
    
    response = client.chat.completions.create(
      model="gpt4",
      #response_format={ "type": "json_object" },
      messages=messages
    )
    
    match = re.split(r"<[^>]*>",response.choices[0].message.content)
    if len(match) == 5:
        desc = match[1]
        summ = match[3]
        responses.append((desc,summ))
    elif len(match) == 1:
        match = re.split(r"Sjukdomshistoria", txt,maxsplit=1)
        desc = match[0]
        summ = 'Sjukdomshistoria'+match[1]
        
    

0 - 13:07:22
1 - 13:13:33
2 - 13:19:59
3 - 13:23:09
4 - 13:28:04
5 - 13:34:46
6 - 13:35:51
7 - 13:40:52
8 - 13:43:52
9 - 13:47:55
10 - 13:52:06
11 - 13:58:21
12 - 14:04:21
13 - 14:12:55
14 - 14:19:17
15 - 14:22:58
16 - 14:28:48


In [13]:
len(match)

1

In [9]:
response.choices[0].message.content

'```plaintext\nInskrivningsanteckning 24-04-10, Avd 34/12 Kardiologi kl 09.15, Läkare Sara Bergman:\nInskrivningsorsak: Akut myokardinfarkt, genomgången PCI\nTidigare/Nuvarande sjukdomar: Rökning (upphört för 5 år sedan), typ 2-diabetes, behandlad med Metformin. Ingen känd hjärtsjukdom i familjen. Ej känt med några allergier.\nSocialt: Pensionär, tidigare lärare. Gift, tre vuxna barn. Lever ett stillasittande liv.\nAktuella läkemedel: \nTabl Metformin 850 mg 2 x 1 \nTabl Trombyl 75 mg 1 x 1 (påbörjad efter PCI)\nAktuellt: Vaknade tidigt på morgonen med tryck över bröstet och andfåddhet. Efter att ha tvekat tog patienten kontakt med 1177 som rådde omedelbar kontakt med ambulans. Vid ankomst till sjukhuset direktransport till koronarintervention där PCI genomförs framgångsrikt.\nSTATUS:\nAllmäntillstånd: Vaken, RLS 1, trött men orienterad.\nHjärta: Lätt tackykard, men rytmisk och utan extraljud.\nLunga: Normalt andningsljud, ingen andnöd i vila.\nBedömning/åtgärd: 68-årig patient med tid

In [15]:
df = pd.DataFrame(responses,columns=["description","summary"])
df.to_parquet("synthetic_bertil.parquet",compression=None)

In [20]:
df.iloc[0]["description"]

'{"namn": "Eva Ekvärd", "ålder": 72, "kön": "kvinna", "bakgrund": "Pensionerad bibliotekarie. Änka sedan 5 år tillbaka.", "medicinsk_historik": "Behandlad för bröstcancer för 8 år sedan. Well-controlled diabetes typ 2. Morsan hade osteoporos.", "medicinering": ["Metformin 500 mg två gånger dagligen", "Letrozol 2.5 mg dagligen"], "allergier": "Allergisk mot penicillin"}Aktuellt:\nPatient söker efter upprepad bröstsmärta och har varit orolig för återfall av bröstcancer. Ett rutinmammografi visade ingen återkomst av cancer, men en DEXA-skanning bekräftade osteoporos.\n\nPatienten är rekommenderad att påbörja behandling med bisfosfonater samt kalcium och vitamin D-tillskott.\n\nDiabetes typ 2 fortsätter att vara välkontrollerad.\n'

In [21]:
df.iloc[0]["summary"]

'*Sjukdomshistoria*:\nBröstcancer (fri sedan 8 år), diabetes typ 2, osteoporos (nydiagnostiserad)\n\n*Sökorsaker*:\nBröstsmärta och oro för cancer återfall\n\n*Åtgärder*:\nInled behandling för osteoporos (bisfosfonater, kalcium, vitamin D)\n'

In [23]:
%pwd

'/workspace/DATX05/OpenAI'

In [56]:
df_github = pd.read_csv("../Data/Fine-tuning/dataset.csv",sep=";")
df_github.drop(columns=['id', 'larm','_validation_status'], inplace=True)
df_github.head()

,sokorsak,kon,alder,socialt,tidigare_sjukdomar,aktuellt,hereditet,rokning,allmantillstand,hjarta,...,A,B,C,D,E,lab,ekg,bedomning,planering,huvuddiagnos_001
0,Dyspné,K,58,"Gift, tre utflugna barn, arbetar som undersköt...","Hypertoni, hypotyreos, hyperlipidemi",Förkyld för tio dagar sedan. Efter en vecka fö...,Far avled i hjärtinfarkt vid 53 års ålder.,"Ja, ett paket/dag sedan 15 åå.",NaN,NaN,...,"A fritt, ingen stridor. Tom munhåla. Ingen mis...","AF 35, pox 96% på 10 liter syrgas. Ausk pulm v...","Blodtryck 80/60, puls 110. KÅF 4 sek. Ausk cor...","RLS 1-2, likstora pupiller som reagerar för di...","Temp 39,7. Inspekterar hudkostymen utan petech...",NaN,Sinustakykardi utan tecken till ischemi.,"Sepsis, troligen på basis pneumoni. Erhåller v...",Inläggning. Kontaktar narkosjour för iva-bedöm...,NaN
1,Dyspné,K,77,Ensamboende utan SHT,Hypertoni. KOL. Tidigare hjärtinfarkt. Hjärtsv...,Tilltagande dyspné senaste veckan som akutiser...,NaN,Slutade för 10 år sedan i samband med hjärtinf...,Lätt vilodyspné. Afebril.,Regelbunden rytm utan hörbara bi- eller blåsljud.,...,NaN,NaN,NaN,NaN,NaN,"Hb, vita, CRP ua. TnI lätt förhöjt såsom tidig...",Inferior Q-våg såsom tidigare. Inga tecken til...,Lätt obstruktiv och får inhalera. Ultraljud be...,NaN,I50.9
2,BrastSmarta,M,51,Gift. 2 barn. Golvläggare.,Hypertoni. Typ 2 diabetes.,Inkommer med några dagars tilltagande retroste...,Mor med MI i 60-årsåldern.,Slutat för 5 år sedan. 10 paketår.,Opåverkad.,Regelbunden rytm. Inga hörbara bi- eller blåsl...,...,NaN,NaN,NaN,NaN,NaN,Venös gas ua. CRP 6. LPK 12. Tnt 60-90-300,Sinusrytm med frekvens 70/min. Inga st-höjning...,Tilltagande retrosternal brsm i samband med an...,Inläggning. Erhåller full AKS-behandling. Avst...,I21.9
3,BukSmarta,K,20,Läser till socionom,NaN,För 2 dagars sedan debut av lågt sittande buks...,NaN,NaN,Feber,Normalt,...,NaN,NaN,NaN,NaN,NaN,CRP 68. Vita 15. U-sticka 2 + vita. u-hCG neg.,NaN,Misstänkt appendicit. Fastande sedan kl 19 igå...,NaN,K379 Appendicit
4,BukSmarta,K,78,Änka. ensamboende utan hemtjänst. Tidigare arb...,Tablettbehandlad hypertoni. Känd divertikulos ...,Sedan en vecka lågt sittande buksmärta samt lå...,NaN,NaN,SUbfebril.,Normalt,...,NaN,NaN,NaN,NaN,NaN,"CRP 108. Vita 14,4. U-sticka 1 + vita, 1+ prot...",Normalt,"Recidiverande divertikuliter, nu med fortsatt ...",NaN,K573 Divertikulit


In [57]:
instructions_github = """You are a helpful medical assistant who helps medical proffesionals by summarizing information about patients.
Answer with a bulleted list for each category in the given template.
Answer in Swedish.
"""

In [58]:
responses_github = []

In [ ]:
for i,row in df_github.iterrows():
    txt = ""
    #row.dropna(inplace=True)
    #print(dir(row))
    now = datetime.now()
    one_hour_later = now + timedelta(hours=1)
    current_time = one_hour_later.strftime("%H:%M:%S")
    
    print(i,"-",current_time)
    
    for col in df_github.keys():
        row[col] = str(row[col])
        if row[col] and row[col] != "nan":
            txt += f"{col}: {row[col]} \n"
            
    
    question = f"""
    Below is the history of a patient during one day
    <anamnesis>
    {txt}
    </anamnesis>
    You must select information that fits in the template below. Avoid unnecessary information and only pick out things that relate to each heading. If relevant information is missing, leave the heading blank. Do not include any information in multiple headings.
    <template>
    *Sjukdomshistoria (The patient's diagnoses, medical history and risk factors (e.g. diseases in the family))*

    *Sökorsaker (Patient's symptoms and/or date of procedure)*

    *Åtgärder (Planned examinations, treatments and measures)*

    </template>"""
    
    messages = [{"role": "system", "content": instructions}, {"role": "user", "content": question}]
    
    response = client.chat.completions.create(
      model="gpt4",
      #response_format={ "type": "json_object" },
      messages=messages
    )
    
   
    responses_github.append((txt,response.choices[0].message.content))
    
    
    

0 - 15:39:50
1 - 15:42:38
2 - 15:45:22
3 - 15:50:07
4 - 15:50:56
5 - 15:57:22
6 - 16:01:16
7 - 16:02:12
8 - 16:09:26


In [76]:
responses_github[0][1]

'*Sjukdomshistoria*\nHypertoni, hypotyreos, hyperlipidemi. Far avled i hjärtinfarkt vid 53 års ålder. Rökare, ett paket/dag sedan 15 åå.\n\n*Sökorsaker*\n58-årig kvinna med dyspné. Förkyld för tio dagar sedan, blev sedermera allt tröttare och frossig. Temp 39,1 grader under kvällen. Andades snabb och talade osammanhängande. Förnekar bröstsmärta, buksmärta, trängningar och sveda vid miktion. \n\n*Åtgärder*\nSyrgas på mask 10 liter. Iv. vätska; totalt tre liter Ringer. Blod- och urinodlingar. Antibiotika med Cefotaxim och engångsdos Gensumycin iv. Inläggning och kontakt med narkosjour för iva-bedömning. Karens på blodtrycksmedicin.'

In [79]:
len(responses_github)

51

In [80]:
responses_github[0]

('sokorsak: Dyspné \nkon: K \nalder: 58 \nsocialt: Gift, tre utflugna barn, arbetar som undersköterska. \ntidigare_sjukdomar: Hypertoni, hypotyreos, hyperlipidemi \naktuellt: Förkyld för tio dagar sedan. Efter en vecka förbättrad och gick tillbaka till arbetet. Sedan två dagar allt tröttare och frossig. Kom hem från jobbet och blev liggandes i en soffa hela kvällen. Temp 39,1 grader med rektaltermometer. Sov dåligt, svettades och blev allt mer tungandad. Imorse svårväckt i sängen när maken försökte väcka henne. Andades snabbt och talade osammanhängande. Maken blev orolig och ringde 1177 som kopplade vidare samtalet till ambulansen. Patienten har själv svårt att ge någon utförligare anamnes men förnekar bröstsmärta, tryck över bröstet och utstrålning i arm eller käke. Förnekar även buksmärta, trängningar och sveda vid miktion. Senaste avföring tre dagar sedan, då ua. Ej kunnat äta eller dricka sedan igår morse. \nhereditet: Far avled i hjärtinfarkt vid 53 års ålder. \nrokning: Ja, ett p

In [107]:
df_github = pd.DataFrame(responses_github, columns=["description","summary"])
df_github

,description,summary
0,sokorsak: Dyspné \nkon: K \nalder: 58 \nsocial...,"*Sjukdomshistoria*\nHypertoni, hypotyreos, hyp..."
1,sokorsak: Dyspné \nkon: K \nalder: 58 \nsocial...,<Sjukdomshistoria>\n58-årig kvinnlig patient m...
2,sokorsak: Dyspné \nkon: K \nalder: 77 \nsocial...,*Patientens namn och personnummer är fingerade...
3,sokorsak: BrastSmarta \nkon: M \nalder: 51 \ns...,"*Sjukdomshistoria*\nHypertoni, Typ 2 diabetes...."
4,sokorsak: BukSmarta \nkon: K \nalder: 20 \nsoc...,*Sjukdomshistoria*\nMisstänkt appendicit (K379...
5,sokorsak: BukSmarta \nkon: K \nalder: 78 \nsoc...,*Sjukdomshistoria*\nHypertoni (tablettbehandla...
6,sokorsak: Arytmi \nkon: K \nalder: 31 \nsocial...,*Patientens namn har genererats syntetiskt och...
7,sokorsak: HuvudSmarta \nkon: K \nalder: 23 \ns...,"*Sjukdomshistoria*\nFrisk och medicinfri, tidi..."
8,sokorsak: Bortfall \nkon: K \nalder: 57 \nsoci...,*Sjukdomshistoria*\nTablettbehandlad hypertoni...
9,sokorsak: HuvudSkada \nkon: M \nalder: 81 \nso...,*Sjukdomshistoria*\nHuvudskada. Mannen är 81 å...


In [108]:
df_github.to_parquet("synthetic_data_github.parquet")